In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm



Define the main data set here 

In [10]:
data = {
    "Year": [2017,2018,2019,2020,2021,2022,2023,2024,2025],
    "Population": [648251,661977,674913,687725,699253,692769,685476,684994,662284],
    "SplashPads": [63,61,71,71,79,84,86,86,100],
    "SkateParks": [2,2,3,3,3,4,5,5,5],
    "DogParks": [7,8,12,13,13,10,12,12,12],
    "TennisCourts": [78,91,91,97,97,135,138,139,152],
    "Pools": [24,24,20,20,21,29,28,30,30],
    "RecCenters": [31,31,31,31,37,37,36,39,39]
}

df = pd.DataFrame(data)


In [11]:
df["Year_c"] = df["Year"] - df["Year"].min()


Fit Poisson regression here for the main data

In [12]:
def fit_poisson(y_col):
    X = sm.add_constant(df["Year_c"])
    y = df[y_col]
    
    model = sm.GLM(
        y,
        X,
        family=sm.families.Poisson(),
        offset=np.log(df["Population"])
    )
    
    return model.fit()


Here we fit Poissoin modes and run through the model for each predictor so the "facility_cos"

In [14]:
facility_cols = [
    "SplashPads",
    "SkateParks",
    "DogParks",
    "TennisCourts",
    "Pools",
    "RecCenters"
]

models = {col: fit_poisson(col) for col in facility_cols}


Forecasts here

In [15]:
print(models["SplashPads"].summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:             SplashPads   No. Observations:                    9
Model:                            GLM   Df Residuals:                        7
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -28.511
Date:                Sun, 01 Feb 2026   Deviance:                       1.3679
Time:                        13:55:31   Pearson chi2:                     1.37
No. Iterations:                     4   Pseudo R-squ. (CS):             0.7770
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -9.2997      0.075   -123.571      0.0

In [16]:
future_years = np.arange(2026, 2036)

future_df = pd.DataFrame({
    "Year": future_years,
    "Year_c": future_years - df["Year"].min(),
    "Population": df.loc[df["Year"] == 2025, "Population"].values[0]
})


In [17]:
X_future = sm.add_constant(future_df["Year_c"])
offset_future = np.log(future_df["Population"])

predictions = pd.DataFrame({"Year": future_years})

for col, model in models.items():
    predictions[col] = model.predict(X_future, offset=offset_future)


In [18]:
predictions

,Year,SplashPads,SkateParks,DogParks,TennisCourts,Pools,RecCenters
0,2026,98.863984,6.102358,13.118687,163.328910,30.024450,39.512999
1,2027,104.395910,6.900919,13.668867,177.448331,31.300655,40.778455
2,2028,110.237374,7.803980,14.242121,192.788345,32.631105,42.084439
3,2029,116.405697,8.825216,14.839416,209.454468,34.018106,43.432248
4,2030,122.919168,9.980093,15.461761,227.561341,35.464063,44.823223
5,2031,129.797099,11.286098,16.110207,247.233513,36.971481,46.258745
6,2032,137.059885,12.763008,16.785847,268.606300,38.542972,47.740242
7,2033,144.729059,14.433188,17.489823,291.826717,40.181261,49.269186
8,2034,152.827361,16.321930,18.223322,317.054487,41.889186,50.847097
9,2035,161.378803,18.457834,18.987584,344.463142,43.669707,52.475542
